# 5G Network Operations Insights with Fine Tuning of T5-Small (This is the smallest version of T5.)
## Project Overview
Author: Fatih E. NAR<br>
This project aims to deliver a 5g network insight with fine tuning a network performant encoder-decoder TransformerNN<br>

In [1]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import lzma
import shutil
import pandas as pd
import os
import torch
import threading
import sys
import time
import gc
from datasets import Dataset
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, get_linear_schedule_with_warmup, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import get_peft_model, LoraConfig, TaskType

# Save the model and tokenizer
model_save_path = "models/5g_oss_model"
model_name = "t5-small"

# Set TOKENIZERS_PARALLELISM to false to avoid warnings
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Clear GPU cache before starting
torch.cuda.empty_cache()
gc.collect()

# Cap memory usage to a specific size (e.g., 8 GB) for cuda
max_memory_gb = 8
max_memory_mb = max_memory_gb * 1024
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = f'max_split_size_mb:{max_memory_mb}'

# Check if any accelerator is available 
if torch.cuda.is_available():
    print("Using CUDA (NVIDIA GPU)")
    os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
    # Leverage multi-gpu if available
    device = torch.device("cuda")
    print("Using CUDA")
    # Clear GPU cache before starting
    torch.cuda.empty_cache()
# Check if MPS (Apple Silicon GPU) is available
elif torch.backends.mps.is_available():
    os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
    os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
    # Leverage multi-gpu if available
    device = torch.device("mps")
    print("Using MPS")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Extract the .xz file
with lzma.open('data/5G_netops_data_1M.csv.xz', 'rb') as f_in:
    with open('data/5G_netops_data_1M.csv', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Load the synthetic telecom data
data_path = "data/5G_netops_data_1M.csv"
data = pd.read_csv(data_path)

# Display basic information about the full dataset
data.info()
data.head()

/Users/fenar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/fenar/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/fenar/Library/Python/3.9/lib/python/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/thread

Using MPS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 27 columns):
 #   Column                                Non-Null Count    Dtype  
---  ------                                --------------    -----  
 0   Season                                1000000 non-null  object 
 1   Cell Availability (%)                 1000000 non-null  float64
 2   MTTR (hours)                          1000000 non-null  float64
 3   Throughput (Mbps)                     1000000 non-null  float64
 4   Latency (ms)                          1000000 non-null  float64
 5   Packet Loss Rate (%)                  1000000 non-null  float64
 6   Call Drop Rate (%)                    1000000 non-null  float64
 7   Handover Success Rate (%)             1000000 non-null  float64
 8   Alarm Count                           1000000 non-null  int64  
 9   Critical Alarm Count                  1000000 non-null  int64  
 10  Parameter Changes                     1000000

,Season,Cell Availability (%),MTTR (hours),Throughput (Mbps),Latency (ms),Packet Loss Rate (%),Call Drop Rate (%),Handover Success Rate (%),Alarm Count,Critical Alarm Count,...,Security Incidents,Authentication Failures,Temperature (°C),Humidity (%),Weather,Issue Reported,City,State,Zip,Fault Occurrence Rate
0,Fall,95.7495,5.81,18.7360,92.3200,1.6300,1.64255,96.65,5,2,...,1,8,16.82,38.56,Clear,no,Port Jennifer,VA,43568,2
1,Winter,99.3145,7.91,-19.6452,61.2490,1.2054,2.78315,95.64,6,1,...,4,0,38.52,53.13,Clouds,no,East John,WA,56449,1
2,Winter,92.2955,1.89,138.7740,74.4800,0.0100,1.97045,91.90,4,1,...,4,7,19.21,55.48,Clear,yes,Andreview,PR,77788,2
3,Summer,97.4210,1.58,-126.9951,46.8395,5.1783,0.18300,97.49,4,0,...,2,7,24.83,87.41,Clouds,no,Stacybury,GU,21375,2
4,Summer,97.2150,1.10,29.3080,38.1225,3.2718,1.04080,92.47,3,0,...,1,9,37.49,61.02,Clouds,no,East Laurastad,AR,4893,1


In [3]:
# Fill NaN values and prepare input and target texts
# Ensure all NaN values are filled with empty strings
data = data.fillna('')

# Ensure 'Zip' column is treated as a string
data['Zip'] = data['Zip'].astype(str)

# Prepare the input_text and target_text columns
data['input_text'] = data.apply(lambda row: f"Season: {row['Season']} Cell Availability: {row['Cell Availability (%)']} MTTR: {row['MTTR (hours)']} Throughput: {row['Throughput (Mbps)']} Latency: {row['Latency (ms)']} Packet Loss Rate: {row['Packet Loss Rate (%)']} Call Drop Rate: {row['Call Drop Rate (%)']} Handover Success Rate: {row['Handover Success Rate (%)']} Alarm Count: {row['Alarm Count']} Critical Alarm Count: {row['Critical Alarm Count']} Parameter Changes: {row['Parameter Changes']} Successful Configuration Changes: {row['Successful Configuration Changes (%)']} Data Usage: {row['Data Usage (GB)']} User Count: {row['User Count']} Signal Strength: {row['Signal Strength (dBm)']} Jitter: {row['Jitter (ms)']} Connection Setup Success Rate: {row['Connection Setup Success Rate (%)']} Security Incidents: {row['Security Incidents']} Authentication Failures: {row['Authentication Failures']} Temperature: {row['Temperature (°C)']} Humidity: {row['Humidity (%)']} Weather: {row['Weather']} Issue Reported: {row['Issue Reported']} City: {row['City']} State: {row['State']} Zip: {row['Zip']}", axis=1)
data['target_text'] = data['Fault Occurrence Rate'].astype(str)


# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(data)

# Split the dataset into training and evaluation
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Check the loaded dataset
print(f"Training Dataset size: {len(train_dataset)}")
print(f"Evaluation Dataset size: {len(eval_dataset)}")
print(train_dataset[0])

Training Dataset size: 800000
Evaluation Dataset size: 200000
{'Season': 'Fall', 'Cell Availability (%)': 90.865, 'MTTR (hours)': 7.5, 'Throughput (Mbps)': -9.001500000000002, 'Latency (ms)': 68.77, 'Packet Loss Rate (%)': 3.6408, 'Call Drop Rate (%)': 2.0601, 'Handover Success Rate (%)': 96.74, 'Alarm Count': 11, 'Critical Alarm Count': 2, 'Parameter Changes': 12, 'Successful Configuration Changes (%)': 99.68, 'Data Usage (GB)': 31.68, 'User Count': 4873, 'Signal Strength (dBm)': -66.96, 'Jitter (ms)': 26.9, 'Connection Setup Success Rate (%)': 99.96, 'Security Incidents': 4, 'Authentication Failures': 8, 'Temperature (°C)': 34.51, 'Humidity (%)': 86.18, 'Weather': 'Clouds', 'Issue Reported': 'yes', 'City': 'Smithville', 'State': 'OK', 'Zip': '54923', 'Fault Occurrence Rate': 3, 'input_text': 'Season: Fall Cell Availability: 90.865 MTTR: 7.5 Throughput: -9.001500000000002 Latency: 68.77 Packet Loss Rate: 3.6408 Call Drop Rate: 2.0601 Handover Success Rate: 96.74 Alarm Count: 11 Critic

In [4]:
# Load the tokenizer from the pretrained model
tokenizer = T5Tokenizer.from_pretrained(model_name)
# Match Tokenizer to the model
tokenizer.add_tokens([f'<SPL_{i}' for i in range(0,28)])
# Add the pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
else:
    tokenizer.pad_token = tokenizer.eos_token
#print len of tokenizer
print(len(tokenizer))

# Save the tokenizer
tokenizer.save_pretrained(model_save_path)
model = T5ForConditionalGeneration.from_pretrained(model_name)


# Tokenize datasets
def preprocess_function(examples):
    inputs = examples['input_text']
    targets = examples['target_text']
    model_inputs = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, padding='max_length', truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

columns = ['input_ids', 'attention_mask', 'labels']
train_dataset.set_format(type='torch', columns=columns)
eval_dataset.set_format(type='torch', columns=columns)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


32128


Map:   0%|          | 0/800000 [00:00<?, ? examples/s]/Users/fenar/Library/Python/3.9/lib/python/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 200000/200000 [02:48<00:00, 1188.67 examples/s]


In [11]:
# Define PEFT/LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=4, # it was 2
    lora_alpha=32, # it was 16
    lora_dropout=0.1, # it was 0.05
    target_modules=['q', 'v', 'k', 'o']
)
model = get_peft_model(model, lora_config)
model.to(device)  # Language modeling head to GPU 
model.eval()

/Users/fenar/Library/Python/3.9/lib/python/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 512)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 512)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=512, out_features=512, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=512, out_features=4, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=4, out_features=512, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
               

In [12]:
# Set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  # Output directory
    overwrite_output_dir=True,  # Overwrite the content of the output directory
    num_train_epochs=5,  # Number of training epochs
    per_device_train_batch_size=42,  # Batch size per device during training
    gradient_accumulation_steps=7,  # Accumulate gradients over multiple steps
    learning_rate=5e-5,  # Learning rate
    save_steps=2000,  # Save checkpoint every 2000 steps
    save_total_limit=2,  # Limit the total amount of checkpoints
    eval_strategy="steps",  # Evaluate during training at each `logging_steps`
    logging_steps=500,  # Log every 500 steps
    eval_steps=2000,  # Evaluate every 2000 steps
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="loss",  # Use loss to evaluate the best model
    predict_with_generate=True,  # Use generation for evaluation
    fp16=False,  # Disable mixed precision training for MPS
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

  4%|▎         | 500/13605 [54:44<23:52:57,  6.56s/it]

{'loss': 0.3141, 'grad_norm': 0.018248245120048523, 'learning_rate': 4.816244027930908e-05, 'epoch': 0.18}


  7%|▋         | 1000/13605 [1:49:30<22:57:14,  6.56s/it]

{'loss': 0.0152, 'grad_norm': 0.006048723589628935, 'learning_rate': 4.6324880558618154e-05, 'epoch': 0.37}


 11%|█         | 1500/13605 [2:44:08<22:02:06,  6.55s/it]

...

 99%|█████████▉| 13500/13605 [30:44:59<11:56,  6.82s/it]  

{'loss': 0.0, 'grad_norm': nan, 'learning_rate': 3.858875413450937e-07, 'epoch': 4.96}


100%|██████████| 13605/13605 [30:56:55<00:00,  8.19s/it]

{'train_runtime': 111415.7272, 'train_samples_per_second': 35.902, 'train_steps_per_second': 0.122, 'train_loss': 0.02188537514240415, 'epoch': 5.0}


TrainOutput(global_step=13605, training_loss=0.02188537514240415, metrics={'train_runtime': 111415.7272, 'train_samples_per_second': 35.902, 'train_steps_per_second': 0.122, 'total_flos': 5.449646524763996e+17, 'train_loss': 0.02188537514240415, 'epoch': 4.999737505249895})

In [13]:
# Save the model and tokenizer
print(f"Tokenizer Final Size = {len(tokenizer)}")
print(f"Model Final Size = {model.get_input_embeddings().weight.shape[0]}")
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print("Training complete and model saved.")

Tokenizer Final Size = 32128
Model Final Size = 32128
Training complete and model saved.


/Users/fenar/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
# Results
results = trainer.evaluate(eval_dataset)
print("Evaluation Results:", results)

100%|██████████| 25000/25000 [58:03<00:00,  7.18it/s]  

Evaluation Results: {'eval_loss': 0.013824737630784512, 'eval_runtime': 3483.8647, 'eval_samples_per_second': 57.408, 'eval_steps_per_second': 7.176, 'epoch': 4.999737505249895}
